In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
def ingest_train():
#     data = pd.read_csv('/kaggle/input/sentiment-analysis-dataset-100000/Sentiment Analysis Dataset 100000.csv', encoding='latin-1')
    data = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
#     data = data[data.Sentiment.isnull() == False]
    data = data[data.sentiment.isnull() == False]
    data['Sentiment'] = data['sentiment']
    data['SentimentText'] = data['review']
#     data = data[data['SentimentText'].isnull() == False]
    data = data[data['review'].isnull() == False]
    data.reset_index(inplace=True)
    data.drop('index', axis=1, inplace=True)
    return data
train = ingest_train()
train.head()

In [ ]:
import re
pat_1 = r"(?:\@|https?\://)\S+"
pat_2 = r'#\w+ ?'
ascii = r'[^\x00-\x7F]+'
combined_pat = r'|'.join((pat_1, pat_2))
www_pat = r'www.[^ ]+'
html_tag = r'<[^>]+>'
negations_ = {"isn't":"is not", "can't":"can not","couldn't":"could not", "hasn't":"has not",
                "hadn't":"had not","won't":"will not","wouldn't":"would not","aren't":"are not",
                "haven't":"have not", "doesn't":"does not","didn't":"did not","n't":"not",
                "don't":"do not","shouldn't":"should not","wasn't":"was not", "weren't":"were not",
                "mightn't":"might not","mustn't":"must not"}
negation_pattern = re.compile(r'\b(' + '|'.join(negations_.keys()) + r')\b')

In [ ]:
def data_cleaner(text):
    try:
        stripped = re.sub(combined_pat, '', text)
        stripped = re.sub(www_pat, '', stripped)
        cleantags = re.sub(html_tag, '', stripped)
        rm_non_ascii = re.sub(ascii, '', cleantags)
        lower_case = rm_non_ascii.lower()
        neg_handled = negation_pattern.sub(lambda x: negations_[x.group()], lower_case)
        letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
        return letters_only
    except:
        return 'NC'

In [ ]:
import numpy as np
train['Text_Clean'] = np.vectorize(data_cleaner)(train['SentimentText'])

In [ ]:
from nltk import word_tokenize
tokens = [word_tokenize(sen) for sen in train.Text_Clean]

In [ ]:
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

In [ ]:
def remove_stop_words(tokens): 
    return [word for word in tokens if word not in stoplist]

filtered_words = [remove_stop_words(sen) for sen in tokens]

In [ ]:
from nltk.stem import PorterStemmer

porter = PorterStemmer()

stem =[]
for sentence in filtered_words:
    stem.append(["".join(porter.stem(word)) for word in sentence])

In [ ]:
result = [' '.join(sen) for sen in stem]

In [ ]:
train['Text_Final'] = result
train['Tokens'] = stem

In [ ]:
train.Sentiment.replace('positive',1,inplace=True)
train.Sentiment.replace('negative',0,inplace=True)

In [ ]:
train = train[['Text_Final', 'Tokens', 'Sentiment']]
train.head()

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from wordcloud import WordCloud, STOPWORDS


neg_tweets = train[train.Sentiment == 0]
neg_string = []
for t in neg_tweets.Text_Final:
    neg_string.append(t)
neg_string = pd.Series(neg_string).str.cat(sep=' ')
from wordcloud import WordCloud

wordcloud = WordCloud(width=1600, height=800,max_font_size=200, background_color='white').generate(neg_string)
plt.figure(figsize=(12,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
pos_tweets = train[train.Sentiment == 1]
pos_string = []
for t in pos_tweets.Text_Final:
    pos_string.append(t)
pos_string = pd.Series(pos_string).str.cat(sep=' ')
wordcloud = WordCloud(width=1600, height=800,max_font_size=200,colormap='magma', background_color='white').generate(pos_string)
plt.figure(figsize=(12,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
X = np.array(train.iloc[:,0].values)
y = np.array(train.Sentiment.values)
cv = CountVectorizer(max_features = 1000)
X = cv.fit_transform(train.Text_Final).toarray()
print("X.shape = ",X.shape)
print("y.shape = ",y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
# split the dataset in train and test
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.10,random_state=42)
print("Train shapes : X = {}, y = {}".format(x_train.shape,y_train.shape))
print("Test shapes : X = {}, y = {}".format(x_test.shape,y_test.shape))

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(penalty='l2', C=c, max_iter=7600, random_state=42)
    lr.fit(x_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_test, lr.predict(x_test))))

In [ ]:
final_model = LogisticRegression(penalty='l2', C=0.25, max_iter=7600, random_state=42)
final_model.fit(x_train, y_train)
print ("Final Accuracy: %s" 
       % accuracy_score(y_test, final_model.predict(x_test)))

In [ ]:
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
    )
}
print("POSITIVE")
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:5]:
    print (best_positive)
print("NEGATIVE")
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:5]:
    print (best_negative)

In [ ]:
from sklearn.metrics import confusion_matrix, roc_curve,  roc_auc_score, classification_report

In [ ]:
#F1 Score, Recall and Precision
target_names=['Positive', 'Negative']
y_pred = final_model.predict(x_test)
print(classification_report(y_test, y_pred, target_names=target_names))

In [ ]:
import matplotlib.pyplot as plt
#ROC AUC curve
rocAuc = roc_auc_score(y_test, y_pred)

falsePositiveRate, truePositiveRate, _ = roc_curve(y_test, y_pred)

plt.figure()

plt.plot(falsePositiveRate, truePositiveRate, color='green',
         lw=3, label='ROC curve (area = %0.2f)' % rocAuc)
plt.plot([0, 1], [0, 1], color='navy', lw=3, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic of Sentiment Analysis Model')
plt.legend(loc="lower right")
plt.show()

In [ ]:
#Other accuracy metrices
y_pred = (y_pred > 0.5)

#confusion metrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
import seaborn as sn
sn.set(font_scale=1.4) # for label size
x_axis_labels = y_axis_labels = target_names
sn.heatmap(cm, annot=True, annot_kws={"size": 16}, fmt='g', xticklabels=x_axis_labels, yticklabels=y_axis_labels)
plt.show()

In [ ]:
# movie reviews from IMDB website, Jocker Movie (2019)
t_1 = "This is a movie that only those who have felt alone and isolated can truly relate to it. You understand the motive and you feel sorry for the character. A lot of people will see this movie and think that it encourages violence. But truly, this movie should encourage each and every one of us to become a better person, treat everyone with respect and make each other feel like they belong in this world, instead of making them feel isolated."
t_2 = "Truly a masterpiece, The Best Hollywood film of 2019, one of the Best films of the decade... And truly the Best film to bring a comic book so chillingly and realistically to real ife. Remarkable Direction, Cinematography, Music and the Acting. Some people are surprised to find it DISTURBING and VIOLENT, but it's a necessity and message. It's about society and reflects those underappreciated/unrecognized/bullied people, proving they can do something too. The way it shows class difference, corruption and how rich and talented rule others around them is not exaggerated and that's what makes it different. It's BELIEVABLE. There could be multiple JOKERs living in our society that could shake those around them in much bitter way than the film shows making people uncomforting people. Consider this a wake up call, a message, but first a film. A PERFECT film."
t_3 = "Joaquin Phoenix gives a tour de force performance, fearless and stunning in its emotional depth and physicality. It's impossible to talk about this without referencing Heath Ledger's Oscar-winning performance from The Dark Knight, widely considered the definitive live-action portrayal of the Joker, so let's talk about it. The fact is, everyone is going to be stunned by what Phoenix accomplishes, because it's what many thought impossible - a portrayal that matches and potentially exceeds that of The Dark Knight's Clown Prince of Crime"
t_4 = "Most of the time movies are anticipated like this they end up falling short, way short. Joker is the first time I was more than happy with the hype. Please ignore the complaints of 'pernicious violence' as they are embarrassing to say the least. We haven't seen a comic movie this real before. If we ever 'deserved' a better class of criminal - Phillips and Phoenix have delivered. This is dark, Joker IS dark and you will fall in love with the villain as you should. The bad guys are always more romantic anyway."
t_5 = "Let me start off by saying if Joaquin Phoneix doesn't get his Oscar for this movie. Then the Oscars should be cancelled. Phoneix is amazing as you might've heard from every review ever! But Todd Phillips. Is at his best here. The story lines he takes and visuals are just breathtaking. The score!! Omg the Score! Every time that score came on, I felt very uncomfortable, like something horrible was about to happen. It was GREAT. The Inspirations from Taxi Driver and King of Comedy are there, and add so much for the movie. And I got to be honest. There are scenes that are very violent. And its disturbing. But I honestly expected it to be WAYYYY more violent from all the controversy going on. Overall this movie was GREAT. Come Oscar season, it needs to be nominated for Best Picture, ScreenPlay, Cinematography, Actor, Score and Director"
t_6 = "I get why some people hate this . It's because of the political message and how some people think that you need get empathy for Arthur's madness. But come on that is not the point and it will never be. Enjoy this masterpiece because Joaquin Phoenix and Todd Phillips overdid themselves with this movie . The acting,music and cinematography are just amazing ! Please enjoy the movie without overthinking it."
t_7 = "It's sad that Joaquin missed Oscar for 'The gladiator' as he was very compelling Villain. But I am quite confident he will win it for the 'Joker'. Damn what a movie !!! Keeps u on the toes all the time. Unpredictable storyline and really a deep interesting plot. Did I forget to mention acting ? Damn Do niro and Joaquin teach u what really is 5 star acting. To enjoy this movie, get a wine in your hand, close the curtains, turn of ur cellphones and put do not disturb sign on ur door. Best Dark thriller suspense movie you will get to experience."
t_8 = "This movie causes the audience to consider many topics on morality, particularly the poor choices made by the Joker. The Joker is more relatable in this movie than antagonists usually are. I think this gives a more realistic view of 'bad' people, who are not usually 100% bad in all ways as society often judges. It is important to know when to sympathize with the Joker (towards the beginning of the movie), and when he needs to be identified as a threat (when he begins making certain bad choices). This movie is only suitable for adults who are willing to pick it apart and think critically about it. It isn't for light amusement. That substance is what makes it so good.If you aren't able to ponder underlying themes on your own, and you have to be handed all your thoughts to you, then this isn't the best movie for you."
t_9 = "I thought this movie was complete and utter garbage. I don't understand why people liked this movie so much. The character doesn't even stay true to the comics and the director flat out said it didn't, and then they made it seem like it was part of Gothams fall and he was just a little boy that had a laughing problem... Hated it"
tests = np.array([t_1, t_2, t_3, t_9]) #, t_4, t_5, t_6, t_7, t_8
X = cv.transform(tests).toarray()
final_model.predict(X)